# Planning processing

In this notebook, we process the edges and vertices previously generated into a `networkx` graph that we can handle locally. 

In [ ]:
import pandas as pd
import networkx as nx
from hdfs3 import HDFileSystem
from time_helpers import compute_delta_time
from planning_helpers import is_transfer_edge

hdfs = HDFileSystem()
group_path = "/user/theAggregators/"

SPEED = 0.05/60 # km/s

## Loading the data

We first load the data from our group folder on hdfs. 

In [ ]:
def read_from_hdfs(file_path):
    '''
        Reads data from the hdfs file system 
        and returns a dataframe corresponding to it
    '''
    result = pd.DataFrame()
    for path in hdfs.ls(file_path):
        if not "SUCCESS" in path:
            with hdfs.open(path) as f:
                result = result.append(pd.read_csv(f, compression='gzip'))
    return result

In [ ]:
# Read the vertice data
vertices = read_from_hdfs(f"{group_path}vertices.csv").drop(columns = ['location_type', 'parent_station']).set_index('stop_id')
vertices.index = vertices.index.astype(str)

# Read the edge data
edges = read_from_hdfs(f"{group_path}delays.csv")

# Read the transfers data
transfers = read_from_hdfs(f"{group_path}transfers.csv")

## Building the graph

We build the networkx graph from the main edge list. We keep the attributes since they will come in useful during the shortest paths search. The graph is a multi-directed, meaning there can be multiple edges between two points, which is what weed need in this case: there might be multiple ways to go from A to B in a city. 

We also add the transfer edges with a route id called `walking`. We set the nodes attributes to contain their localisation and name. 

In [ ]:
# Build the multi-directed graph from the edgelist, retain edge attributes
G = nx.from_pandas_edgelist(edges, source = 'src', target = "dst", edge_attr = True, create_using=nx.MultiDiGraph())

# Add transfer edges to the graph
for idx, row in transfers.iterrows():
    G.add_edge(str(row.src), str(row.dst), distance = row.distance, route_id = "walking")

In [ ]:
# Set nodes attribute that will help display the nodes
nx.set_node_attributes(G, vertices['stop_name'].to_dict(), 'name')
nx.set_node_attributes(G, vertices['stop_lat'].to_dict(), 'lat')
nx.set_node_attributes(G, vertices['stop_lon'].to_dict(), 'lon')

For each of the edges, we compute the time required to traverse it. For transport edges, it is the time at arrival minus the time at departure. For transfer edges, it is the distance divided by the walking speed. 

In [ ]:
# Compute the time to travel through an edge
for u, v, i, data in G.edges(data = True, keys = True):
     # Regular transit edge
    if not is_transfer_edge(G, (u, v, i)):
        t = compute_delta_time(data['src_departure'], data['dst_arrival'])
        nx.set_edge_attributes(G, {(u, v, i): t}, 'travel_time')
    
    # Transfer edge
    else:
        nx.set_edge_attributes(G, {(u, v, i): data['distance']/SPEED}, 'travel_time')

A directed graph is strongly connected if and only if every vertex in the graph is reachable from every other vertex. We check that it is the case, and that Zurich HB is in the largest strongly conected component. 

In [ ]:
print('Is the graph strongly connected ? {}'.format(nx.is_strongly_connected(G)))
print('There are {} strongly connected components in the graph.'.format(nx.number_strongly_connected_components(G)))

In [ ]:
# Show the sizes of all connected components
for i, cc in enumerate(nx.strongly_connected_components(G)):
    print('Component {} has size {}.'.format(i + 1, len(cc)))

In [ ]:
# Take largest connected components and verify that Zurich HB is in it
largest_cc = sorted(nx.strongly_connected_components(G), key=len, reverse=True)[0]
for node in largest_cc:
    if '8503000' in node:
        print('Zurich is in the largest connected component.')
        break

In [ ]:
# Take subgraph corresponding to largest cc
G = G.subgraph(largest_cc).copy()

# Save the graph locally (lfs) 
nx.write_gpickle(G, '../data/graph.pickle')